Esse trabalho visa achar o relacionamento entre dados de um projeto de FPSO finalizado. Atualmente todos esses dados são de entrada manual, uma vez que descobrimos os relacionamentos entre eles é esperado que não seja necessário preencher todos, pois alguns dos atributos poderiam ser preenchidos via query.

In [1]:
pip install xlrd==1.2.0

In [ ]:
pip install openpyxl

In [ ]:
pip install pandas

In [123]:
#Leitura do Arquivo com as queries dos elementos dos projetos, inicialmente leremos somente dos equipamentos na ferramenta SPPID
import pandas as pd
import numpy as np
import os
equip_sppid = pd.read_excel('D:\MBA BI MASTER\TCC\DRGCheck_PF3_REF_2020_10_21_06_08_15.xlsx',index_col=0,skiprows=1,sheet_name='EQUIPMENT_SPPID')
equip_sppid.head()

,TAG,TAG_OK,GROUPTAG,GROUPTAG_OK,PARENTEQPTAG,PARENTTAG_OK,MODULE,DISCIPLINE,FAMILY,TYPE,...,A_Type,A_Type_Of_Driver,A_Ventilation,A_Voltage,A_Voltage_Level,A_WIDTH,A_Zone_0,A_Zone_1,A_Zone_2,TAG_Q
OID,,,,,,,,,,,,,,,,,,,,,
7423637249C04723A0D5726A86C8D724,ACO-5515001,OK,ACO-5515001,OK,NaN,OK,M-18,Telecom,Antenna,ANTENNA COUPLER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9ED9BFFF3037415E8A76EE4BE7E771C6,ACO-5515002,OK,ACO-5515002,OK,NaN,OK,M-13,Telecom,Antenna,ANTENNA COUPLER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7AD6A95492C046088295F5A5060040A9,ACO-5515003,OK,ACO-5515003,OK,NaN,OK,M-15,Telecom,Antenna,ANTENNA COUPLER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2FFB700BCA0E4194B642FEB7CE7D1E76,ACO-5515004,OK,ACO-5515004,OK,NaN,OK,M-18,Telecom,Antenna,ANTENNA COUPLER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
F81CB0BF25CA4D358E3E0DEDBCC94766,ACO-5515005,OK,ACO-5515005,OK,NaN,OK,M-18,Telecom,Antenna,ANTENNA COUPLER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [124]:
#Por se tratar de um projeto finalizado, elementos sem TAG costumam ser lixo, portanto eliminaremos todas as linhas cujo
#TAG = null, de forma a não contaminar o resultado
equip_sppid = equip_sppid.dropna(subset=['TAG'])

In [125]:
#O atributo TAG costuma ser único, porém ele trás informações importantes no pré-fixo, portanto criaremos uma nova coluna 
#somente com o pré-fixo do TAG
equip_sppid.insert (0, "TAG_PREFIX", equip_sppid['TAG'].str.split('-').str[0])
equip_sppid.head()

,TAG_PREFIX,TAG,TAG_OK,GROUPTAG,GROUPTAG_OK,PARENTEQPTAG,PARENTTAG_OK,MODULE,DISCIPLINE,FAMILY,...,A_Type,A_Type_Of_Driver,A_Ventilation,A_Voltage,A_Voltage_Level,A_WIDTH,A_Zone_0,A_Zone_1,A_Zone_2,TAG_Q
OID,,,,,,,,,,,,,,,,,,,,,
7423637249C04723A0D5726A86C8D724,ACO,ACO-5515001,OK,ACO-5515001,OK,NaN,OK,M-18,Telecom,Antenna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9ED9BFFF3037415E8A76EE4BE7E771C6,ACO,ACO-5515002,OK,ACO-5515002,OK,NaN,OK,M-13,Telecom,Antenna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7AD6A95492C046088295F5A5060040A9,ACO,ACO-5515003,OK,ACO-5515003,OK,NaN,OK,M-15,Telecom,Antenna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2FFB700BCA0E4194B642FEB7CE7D1E76,ACO,ACO-5515004,OK,ACO-5515004,OK,NaN,OK,M-18,Telecom,Antenna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
F81CB0BF25CA4D358E3E0DEDBCC94766,ACO,ACO-5515005,OK,ACO-5515005,OK,NaN,OK,M-18,Telecom,Antenna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [126]:
# Os Atributos _OK são atributos de análise de conjunto de outros atributos, portanto não faz sentido utiliar eles na análise
equip_sppid = equip_sppid.filter(regex='^((?!_OK).)*$', axis=1)

In [127]:
# Os Atributos que começam com A_ são apenas de suporte a tabela, não são dados do objeto e devem ser eliminados
equip_sppid = equip_sppid.filter(regex='^((?!^A_).)*$', axis=1)

In [128]:
#As colunas abaixo também são colunas de suporte e não trazem informações relevantes
equip_sppid = equip_sppid.drop(['STATUS_COMPLETENESS', 'KEYFIELD_TOCOMPARE','DISCIPLINE_FAMILY','DISCIPLINE_FAMILY_TYPE','CREATION_ACTOR','MC_TITLE','COMPARISON_SP3D', 'COMPARISON_SPI','CREATION','LAST_MODIFICATION','TAG_Q'], axis=1)

In [129]:
#Em um primeiro momento vamos priorizar trabalhar somente com atributos do tipo texto, pois para atributos numéricos pode
#ser mais interessante trabalhar com correlação e não com árvores de decisão
equip_sppid.columns.to_series().groupby(equip_sppid.dtypes).groups

{dtype('int64'): Index(['QUANTITY'], dtype='object'),
 dtype('float64'): Index(['DRAWING_DISCIPLINE', 'REDUNDANCY_CONFIG', 'SSOP', 'PRESS_DESIGN_MIN',
        'PRESS_DESIGN_MAX', 'TEMP_DESIGN_MIN', 'TEMP_DESIGN_MAX',
        'TEMP_DESIGN_NORMAL', 'TEMP_OPER_NORMAL', 'PRESS_DESIGN_TUBE_NORMAL',
        'TEMP_DESIGN_TUBE_NORMAL', 'PRESS_DESIGN_TUBE_MIN',
        'TEMP_DESIGN_TUBE_MIM', 'PRESS_DESIGN_TUBE_MAX', 'TEMP_DESIGN_TUBE_MAX',
        'PRESS_OPER_TUBE_NORMAL', 'TEMP_OPER_TUBE_NORMAL', 'COMPRESSIBILITY',
        'IMPEDANCE', 'PRIMARY_POWER', 'REACTANCE', 'SHORT_CIRCUIT_POWER',
        'SECONDARY_POWER', 'TERTIARY_POWER', 'FUEL_CONSUMPTION',
        'CONTROL_APPARENT_POWER', 'IP_CODE', 'LINE_DIAMETER', 'RESISTANCE',
        'STATIC_PRESSURE', 'SUB_TRANSIENT_REACTANCE', 'TRANSIENT_REACTANCE',
        'AREA_ADJUST_FACTOR', 'CRITICAL_EQUIPMENT', 'HAS_ELECTRICAL_COMPONENT',
        'PHASES', 'IS_ON_EQUIPMENT_LIST', 'MOUNTING_TYPE', 'NEED_INSULATION',
        'COLD_SIDE_DESIGN_MIN_PRESS'

In [130]:
numeric_columns = set(equip_sppid.select_dtypes(include=['number']).columns)
non_numeric_columns = set(equip_sppid.columns) - numeric_columns

In [131]:
equip_sppid_categoricals = equip_sppid[non_numeric_columns]

In [132]:
equip_sppid_categoricals.nunique()

LENGTH_TAN_TO_TAN                 35
ZONE_0                             1
FREQUENCY_CONVERTER_DRIVEN         2
EQUIP_INSTALLED_IN_OPEN_AREA       2
LIQUID_LEVEL_HIGH                 57
                                ... 
INPUT_VOLTAGE                      7
REMARKS                          743
CONTROL_ACTIVE_POWER              37
DESCRIPTION                     2315
FLAMMABLE_FLUID                    2
Length: 92, dtype: int64

In [133]:
qtd_rows = equip_sppid_categoricals.shape[0]

In [134]:
Columns_distinct = equip_sppid_categoricals.nunique()/qtd_rows

In [135]:
Columns_distinct[Columns_distinct >= 0.10]

DRAWING        0.101518
GROUPTAG       0.721045
TAG            0.989583
REMARKS        0.131179
DESCRIPTION    0.408722
dtype: float64

In [136]:
#Vamos trabalhar somente com colunas cujo a distinção dos valores seja inferior a 70%, pois acima disso significa que são
#atributos de identificação única
equip_sppid = equip_sppid.filter(Columns_distinct[Columns_distinct < 0.10].index, axis=1)

In [137]:
equip_sppid.columns

Index(['LENGTH_TAN_TO_TAN', 'ZONE_0', 'FREQUENCY_CONVERTER_DRIVEN',
       'EQUIP_INSTALLED_IN_OPEN_AREA', 'LIQUID_LEVEL_HIGH', 'SHAFT_POWER',
       'CAPACITY', 'ROTATION', 'PRED_FLUID_IS_COMBUSTIBLE', 'OPER_NORM_PRESS',
       'LIQUID_LEVEL_LOW', 'SHELL_DIAMETER', 'TRANSFCOOLMETHOD',
       'COOLING_CAPACITY', 'CONNECTION_TYPE', 'SUCTION_DESIGN_MAX_PRESS',
       'DIFFERENTIAL_PRESS', 'STEADY_STATE_CURRENT', 'VOLTAGE',
       'OIL_LEVEL_LOW_LOW', 'MODULE', 'HOT_SIDE_DESIGN_MAX_PRESS',
       'HEIGHT_TAN_TO_TAN', 'ZONE_1', 'WIDTH', 'DESIGN_STATUS',
       'DISCHARGE_DESIGN_MAX_PRESS', 'SHORT_CIRCUIT_CURRENT',
       'SECONDARY_VOLTAGE', 'PARENTEQPTAG', 'COOLING_TYPE',
       'HOT_SIDE_DESIGN_MIN_TEMP', 'GRADE', 'HEIGHT', 'TAG_PREFIX',
       'DISPLAYPROPERTY', 'TYPE_OF_DRIVER', 'LEAKAGE_SOURCE',
       'COLD_SIDE_DESIGN_MIN_TEMP', 'AIR_FLOW_RATE', 'VOLTAGE_LEVEL',
       'FLUID_CODE', 'OPER_NORM_FLOW_RATE', 'COLD_SIDE_DESIGN_MAX_TEMP',
       'TOTAL_HEAD', 'HAC_GROUP', 'LENGTH', 'TANK

In [138]:
equip_sppid.shape

(5664, 87)

In [54]:
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
# we are going to look at feature importances so we like putting random features to act as a benchmark.
equip_sppid['rand0'] = np.random.rand(equip_sppid.shape[0])
equip_sppid['rand1'] = np.random.rand(equip_sppid.shape[0])
equip_sppid['rand2'] = np.random.rand(equip_sppid.shape[0])

# testing for relationships.
# for numeric targets.
reg = GradientBoostingRegressor(n_estimators=100, max_depth=5,
                                learning_rate=0.1, loss='ls',
                                random_state=1)
# for categorical targets.
clf = GradientBoostingClassifier(n_estimators=100, max_depth=5,
                                learning_rate=0.1, loss='deviance',
                                random_state=1)

#df_test['calories'] = df_test['calories'].fillna(0) # only calories should have missing values

cols = equip_sppid.columns

# try to predict one feature using the rest of others to test collinearity, so it's easier to interpret the results
for c in cols:
    # c is the thing to predict.
    
    if c not in ['rand0', 'rand1', 'rand2']: 

        y = equip_sppid[c].dropna()
        if y.nunique() > 1:
            X = equip_sppid.dropna(subset=[c])
            X = X.drop([c], axis=1) # drop the thing to predict.
            X = pd.get_dummies(X)

            print(c)

            if c in non_numeric_columns:
                print('O atributo possui as seguintes classes:')
                print('Número de classes:',y.nunique())
                print('Classes',y.unique())
                y = y.astype(str)
                scoring = 'accuracy'
                model = clf
                scores = cross_val_score(clf, X, y, cv=5, scoring=scoring)
                print(scoring + ": %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
            elif c in numeric_columns:
                scoring = 'neg_root_mean_squared_error'
                model = reg
                scores = cross_val_score(reg, X, y, cv=5, scoring=scoring)
                print(scoring.replace('neg_', '') + ": %0.2f (+/- %0.2f)" % (-scores.mean(), scores.std() * 2))
            else:
                print('what is this?')

            model.fit(X, y)
            df_importances = pd.DataFrame(data={'feature_name': X.columns, 'importance': model.feature_importances_}).sort_values(by='importance', ascending=False)
            top10_features = df_importances.iloc[:10]
            print('top 10 features:')
            print(top10_features)

            print()
        else:
            print('O atributo ',c,' possue somente 1 classe : ',y.unique())

LENGTH_TAN_TO_TAN
O atributo possui as seguintes classes:
Número de classes: 35
Classes ['8.92 m' '7.65 m' '7.92 m' '8373 mm' '8.23 m' '7.82 m' '5.74 m' '6.92 m'
 '7.84 m' '7.91 m' '10.2 m' '2.973 m' '7555 mm' '10221 mm' '6.206 m'
 '2.335 m' '4.625 m' '6 m' '3.05 m' '4.05 m' '5.52 m' '7.42 m' '4.97 m'
 '4.8 m' '2.8 m' '2.51 m' '19 m' '9.3 m' '16.0 m' '12 m' '4 m' '4.4 m'
 '8 m' '16.7 m' '11.2 m']


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


accuracy: 0.89 (+/- 0.10)
top 10 features:
               feature_name  importance
15      CAPACITY_1.30E+07 W    0.058127
20         CAPACITY_10.3 MW    0.046587
331    THERMAL_LOAD_16.7 MW    0.042486
139            HEIGHT_1.9 m    0.040544
340     THERMAL_LOAD_6.9 MW    0.029924
140          HEIGHT_1.948 m    0.026415
257         DIAMETER_900 mm    0.023434
62   SHELL_DIAMETER_1250 mm    0.022665
232        DIAMETER_1200 mm    0.020675
126           HEIGHT_1.05 m    0.020299

O atributo  ZONE_0  possue somente 1 classe :  ['-']
FREQUENCY_CONVERTER_DRIVEN
O atributo possui as seguintes classes:
Número de classes: 2
Classes ['No' 'Yes']
accuracy: 0.99 (+/- 0.04)
top 10 features:
                            feature_name  importance
641                         LENGTH_0.4 m    0.492589
859                INSULATION_PURPOSE_NI    0.298443
511                        TAG_PREFIX_UH    0.199271
805                         POWER_230 kW    0.003914
5       EQUIP_INSTALLED_IN_OPEN_AREA_Yes    0.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


accuracy: 0.80 (+/- 0.05)
top 10 features:
                                  feature_name  importance
161                      LIQUID_LEVEL_LOW_HOLD    0.289158
151        LIQUID_LEVEL_LOW_3,73337315518295 m    0.146022
640  LIQUID_LEVEL_HIGH_HIGH_32,1459447197016 m    0.098427
591                 LIQUID_LEVEL_HIGH_HIGH_750    0.041771
643  LIQUID_LEVEL_HIGH_HIGH_32,7850005398475 m    0.039890
507    LIQUID_LEVEL_LOW_LOW_2,00496603493549 m    0.029820
150        LIQUID_LEVEL_LOW_3,68702833602405 m    0.017785
439             SERVICE_MAIN DECK DRAINAGE BOX    0.015319
129                    LIQUID_LEVEL_LOW_0,14 m    0.015303
565                 EQUIP_ORIENTATION_Vertical    0.012692

SHAFT_POWER
O atributo possui as seguintes classes:
Número de classes: 156
Classes ['16.95 kW' '19.54 kW' '6.41 kW' '6.83 kW' '4.00 kW' '19.22 kW' '0.14 kW'
 '24.35 kW' '0.52 kW' '26.9 kW' '15.38 kW' '11.00 kW' '0.42 kW' '0.56 kW'
 '0.28 kW' '8.9 kW' '7.12 kW' '11.65 kW' '13.3 kW' '19.74 kW' '27.2 kW'
 '34

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


accuracy: 0.88 (+/- 0.06)
top 10 features:
                    feature_name  importance
621     AIR_FLOW_RATE_680 m^3/hr    0.260679
576    AIR_FLOW_RATE_1360 m^3/hr    0.049801
639               FLUID_CODE_CNI    0.026717
588    AIR_FLOW_RATE_2040 m^3/hr    0.025076
536                HEIGHT_719 mm    0.024023
452      PARENTEQPTAG_UQ-1261001    0.022053
192      COOLING_CAPACITY_7.0 kW    0.016132
690           TOTAL_HEAD_160.5 m    0.013212
892  SERVICE_SEA WATER LIFT PUMP    0.008956
499               HEIGHT_1350 mm    0.008940

CAPACITY
O atributo possui as seguintes classes:
Número de classes: 457
Classes ['1,00E+01 W' '-' '5,00E+00 W' '109 kW' '124 kW' '1.58E+05 W' '77.7 kW'
 '31.6 kW' '140 kW' '3.5 kW' '163 kW' '7.80E+03 W' '210 kW' '126 kW'
 '21.7 kW' '1.05E+04 W' '1.40E+04 W' '7.00E+03 W' '3.50E+03 W' '79.1 kW'
 '158.4 kW' '56 kW' '70.3 kW' '7 kW' '10.5 kW' '52.5 kW' '255 kW'
 '50.1 kW' '6,00E+00 W' '1,00E+03 W' '4,00E+03 W' '1,00E+00 W'
 '2.83E+07 W' '2100.0 mm' '100 Ah' '31

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [145]:
y

OID
AAAABBBBCCCC00000000000000051295    8.92 m
6827D4F29B644318845A3A04F940D2ED    7.65 m
AAAABBBBCCCC00000000000000051306    8.92 m
AAAABBBBCCCC00000000000000051306    8.92 m
ADC0D8487A3A44158314A752ED78B5FD    7.65 m
                                     ...  
3AB6E3B85EE94895A5F0373E40816474       8 m
AAAABBBBCCCC00000000000000048072       8 m
681BC1FDE9A441ADA31387CB671D8CC0    16.7 m
AAAABBBBCCCC00000000000000048094    11.2 m
AAAABBBBCCCC00000000000000048094    11.2 m
Name: LENGTH_TAN_TO_TAN, Length: 97, dtype: object

In [146]:
X

,rand0,rand1,rand2,ZONE_0_-,EQUIP_INSTALLED_IN_OPEN_AREA_Yes,LIQUID_LEVEL_HIGH_1550,LIQUID_LEVEL_HIGH_1750,LIQUID_LEVEL_HIGH_2000,LIQUID_LEVEL_HIGH_2250,LIQUID_LEVEL_HIGH_2270,...,THERMAL_LOAD_7.5 MW,THERMAL_LOAD_7.8 MW,THERMAL_LOAD_8.29E+06 W,THERMAL_LOAD_8.6 MW,THERMAL_LOAD_8.7 MW,THERMAL_LOAD_809 kW,THERMAL_LOAD_8850 kW,THERMAL_LOAD_9.5 MW,FLAMMABLE_FLUID_No,FLAMMABLE_FLUID_Yes
OID,,,,,,,,,,,,,,,,,,,,,
AAAABBBBCCCC00000000000000051295,0.255761,0.941002,0.003814,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6827D4F29B644318845A3A04F940D2ED,0.192115,0.278450,0.496979,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
AAAABBBBCCCC00000000000000051306,0.613696,0.786036,0.778250,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
AAAABBBBCCCC00000000000000051306,0.362288,0.654040,0.237493,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ADC0D8487A3A44158314A752ED78B5FD,0.237479,0.374000,0.050704,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3AB6E3B85EE94895A5F0373E40816474,0.970152,0.701253,0.235936,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
AAAABBBBCCCC00000000000000048072,0.854311,0.005304,0.422481,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
681BC1FDE9A441ADA31387CB671D8CC0,0.823948,0.924987,0.706413,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [85]:
equip_sppid.apply(lambda x: x.str.replace(' m', '') if x.str[-2:] == ' m' else (x.str.replace(' m', '')/1000 if x.str[-2:] == 'mm' else x))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [117]:
colunas_numericas_a_retirar = []
for c in non_numeric_columns:
    if True in equip_sppid[c].str.contains(r'[0-9]').unique():
        print('Coluna contem numero: ',c)
        print(equip_sppid[c].unique())
        if c not in ['MODULE','ZONE_1','TYPE','ZONE_2','TEMPERATURE_CLASS']:
            colunas_numericas_a_retirar.append(c)
print(colunas_numericas_a_retirar)

Coluna contem numero:  LENGTH_TAN_TO_TAN
[nan '8.92 m' '7.65 m' '7.92 m' '8373 mm' '8.23 m' '7.82 m' '5.74 m'
 '6.92 m' '7.84 m' '7.91 m' '10.2 m' '2.973 m' '7555 mm' '10221 mm'
 '6.206 m' '2.335 m' '4.625 m' '6 m' '3.05 m' '4.05 m' '5.52 m' '7.42 m'
 '4.97 m' '4.8 m' '2.8 m' '2.51 m' '19 m' '9.3 m' '16.0 m' '12 m' '4 m'
 '4.4 m' '8 m' '16.7 m' '11.2 m']
Coluna contem numero:  OPER_NORM_FLOW_RATE
[nan '1200m^3/hr' '10.0 m^3/hr' '650m^3/hr' '80 m^3/hr' '6.3 m^3/hr'
 '60 m^3/hr' '650.0 m^3/hr' '110.0 m^3/hr' '120.0 m^3/hr' '13.5 m^3/hr'
 '10 m^3/hr' '5m^3/hr' '300.0 m^3/hr' '12 m^3/hr' '575.0 m^3/hr'
 '126 m^3/hr' '167.0 m^3/hr' '241.0 m^3/hr' '450.0 m^3/hr' '5 m^3/hr'
 '330.0 m^3/hr' '5.0 m^3/hr' '30 m^3/hr' '15.0 m^3/hr' '800m^3/hr'
 '350.0 m^3/hr' '134.3 m^3/hr' '75 m^3/hr' '530 m^3/hr' '596.7m^3/hr'
 '468.3 m^3/hr' '464.3m^3/hr' '405.6 m^3/hr' '535.1 m^3/hr' '585.3 m^3/hr'
 '441.5 m^3/hr' '594.7 m^3/hr' '536.8 m^3/hr' '364.3 m^3/hr'
 '279.6 m^3/hr' '2400 m^3/hr' '195.0 m^3/hr' '100m^

Coluna contem numero:  FILENAME
['\\1 - Equipment\\1.1 - Black Box System\\Other Parametric Black Box.sym'
 '\\1 - Equipment\\1.1 - Black Box System\\Parametric Black Box.sym'
 '\\7 - HVAC\\7.2 - Equipments\\7.2.1 - Air Conditioning Units\\Fan-Coil Type Air Conditioning Unit.sym'
 '\\7 - HVAC\\7.2 - Equipments\\7.2.1 - Air Conditioning Units\\Fan-Coil Type Air Conditioning Unit with Reheating.sym'
 '\\7 - HVAC\\7.2 - Equipments\\7.2.1 - Air Conditioning Units\\Self-Contained Type Air Conditioning Unit.sym'
 '\\1 - Equipment\\1.1 - Black Box System\\Mechanical Parametric Black Box.sym'
 '\\1 - Equipment\\1.3 - Heat Transfer Equipment\\1.3.1 - Exchangers\\Shell & Tubes.sym'
 '\\1 - Equipment\\1.4 - Mechanical\\1.4.3 - Pumps\\Centrifugal Pump.sym'
 '\\1 - Equipment\\1.4 - Mechanical\\1.4.3 - Pumps\\Metering Pump 4 Heads.sym'
 '\\1 - Equipment\\1.4 - Mechanical\\1.4.3 - Pumps\\Metering Pump 2 Heads.sym'
 '\\1 - Equipment\\1.4 - Mechanical\\1.4.3 - Pumps\\Metering Pump.sym'
 '\\1 - Equipmen

In [116]:
equip_sppid['ROTATION'].unique()

array([nan, '1750 rpm', '2212 rpm', '1750.0 rpm', '1995 rpm', '932 RPM'],
      dtype=object)

In [ ]:
from apyori import apriori

In [15]:
#Alguns atributos estão vazios, pois não são necessário o preenchimento, então está vazio é uma opção de preenchimento,
#para isso vamos substituir nan por EMPTY de forma que vazio seja considerado como uma opção de preenchimento
#equip_sppid = equip_sppid.replace(np.nan, 'EMPTY', regex=True)

In [16]:
equip_sppid_subset = equip_sppid.loc[:,["TAG_PREFIX","DISCIPLINE","FAMILY","TYPE"]]

In [83]:
pip install sklearn

In [84]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
X = equip_sppid_subset.loc[:,["TAG_PREFIX","DISCIPLINE","FAMILY"]]
y = equip_sppid_subset.loc[:,["TYPE"]]

In [85]:
X, y = make_classification(n_samples=1000, n_features=3,n_informative=2, n_redundant=0,random_state=0, shuffle=False)

In [86]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [87]:
clf.fit(X, y)

RandomForestClassifier(max_depth=2, random_state=0)

In [93]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
clf = DecisionTreeClassifier(random_state=1234)
clf.fit(X, y)
text_representation = tree.export_text(clf)
print(text_representation)

|--- feature_1 <= -0.27
|   |--- feature_1 <= -0.46
|   |   |--- feature_0 <= 1.93
|   |   |   |--- feature_1 <= -2.87
|   |   |   |   |--- feature_0 <= -0.26
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_0 >  -0.26
|   |   |   |   |   |--- class: 0
|   |   |   |--- feature_1 >  -2.87
|   |   |   |   |--- feature_1 <= -0.61
|   |   |   |   |   |--- feature_0 <= 0.15
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- feature_0 >  0.15
|   |   |   |   |   |   |--- feature_0 <= 0.15
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- feature_0 >  0.15
|   |   |   |   |   |   |   |--- feature_1 <= -1.19
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- feature_1 >  -1.19
|   |   |   |   |   |   |   |   |--- feature_1 <= -1.19
|   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |   |--- feature_1 >  -1.19
|   |   |   |   |   |   |   |   |   |--- feature_0 <= 0.92
|   |   |   |   |   |   | 

In [101]:
import matplotlib as plt
plt.figure(figsize=(25,20))
tree.plot_tree(clf,feature_names=iris.feature_names,class_names=iris.target_names,filled=True)

TypeError: 'module' object is not callable

In [61]:
pip install h2o


  Created wheel for h2o: filename=h2o-3.32.0.3-py2.py3-none-any.whl size=164649670 sha256=6f9bb3eaea2e5668a5e5b8779d25dd88da27c62a4f322456e2f878df9add20af
  Stored in directory: c:\users\jacks\appdata\local\pip\cache\wheels\e8\fd\ab\49a3ac1befa49ce82776b65bbab34a9870c5bf03e98e56c9ec
Successfully built h2o


In [62]:
import h2o
from h2o.estimators import H2ORandomForestEstimator

In [73]:
predictors = ["TAG_PREFIX"]
response = "DISCIPLINE"

In [72]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.261-b12, mixed mode)
  Starting server from C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\jacks\AppData\Local\Temp\tmp13lkqjwm
  JVM stdout: C:\Users\jacks\AppData\Local\Temp\tmp13lkqjwm\h2o_jacks_started_from_python.out
  JVM stderr: C:\Users\jacks\AppData\Local\Temp\tmp13lkqjwm\h2o_jacks_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.3
H2O_cluster_version_age:,1 month and 5 days
H2O_cluster_name:,H2O_from_python_jacks_85aoj3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.757 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [79]:
pip install pysparkling


  Created wheel for pysparkling: filename=pysparkling-0.6.1-py3-none-any.whl size=187598 sha256=1044cbe7cc39848cb35a4b38b62d5d8148d05d5eb26c898614978ad8372a3e5b
  Stored in directory: c:\users\jacks\appdata\local\pip\cache\wheels\c8\2e\ac\5336e8ec240e9895bf4f142026f16d0d442af578d339c44aa0
Successfully built pysparkling


In [81]:
from pysparkling import *
import h2o
hc = h2oContext.asH2OFrame(equip_sppid_subset)
#hc.as_h2o_frame(equip_sppid_subset)

NameError: name 'h2oContext' is not defined

In [ ]:
train, valid = equip_sppid_subset.split_frame(ratios=[.8], seed=1234)